# Hosting LangGraph Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing LangGraph agent, using Amazon Bedrock AgentCore Runtime. 


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | LangGraph Agents                                                                   |
| Tutorial components | Hosting agent on AgentCore Runtime. Using LangGraph Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |



### Tutorial Key Features

* Hosting LangGraph Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using LangGraph Agents


In [1]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.42.34 requires botocore==1.40.34, but you have botocore 1.40.39 which is incompatible.


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [9]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "langgraph_web_search_agent"
response = agentcore_runtime.configure(
    entrypoint="langgraph_agent_web_search.py",
    execution_role="arn:aws:iam::175918693907:role/_GenAIActivationDay_role",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/langgraph_agent_web_search.py, bedrock_agentcore_name=langgraph_agent_web_search
Configuring BedrockAgentCore agent: langgraph_web_search_agent
Generated Dockerfile: /Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/Dockerfile
Generated .dockerignore: /Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/.dockerignore
Keeping 'langgraph_web_search_agent' as default agent
Bedrock AgentCore configured: /Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/Dockerfile'), dockerignore_path=PosixPath('/Users/sbattoo/Src/AIActivationDay/strands_from_scratch/05-langgraph_agent_web_search-deployto-AgentCore/.dockerignore'), runtime='Docker', region='us-west-2', account_id='175918693907', execution_role='arn:aws:iam::175918693907:role/_GenAIActivationDay_role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [10]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'langgraph_web_search_agent' to account 175918693907 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: langgraph_web_search_agent
✅ ECR repository available: 175918693907.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-langgraph_web_search_agent
Using execution role from config: arn:aws:iam::175918693907:role/_GenAIActivationDay_role
Preparing CodeBuild project and uploading source...


✅ Reusing existing ECR repository: 175918693907.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-langgraph_web_search_agent


Getting or creating CodeBuild execution role for agent: langgraph_web_search_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-1960362299
Reusing existing CodeBuild execution role: arn:aws:iam::175918693907:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-1960362299
Using .dockerignore with 44 patterns
Uploaded source to S3: langgraph_web_search_agent/source.zip
Updated CodeBuild project: bedrock-agentcore-langgraph_web_search_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 10.6s
🔄 DOWNLOAD_SOURCE started (total: 12s)
✅ DOWNLOAD_SOURCE completed in 1.1s
🔄 INSTALL started (total: 13s)
✅ INSTALL completed in 1.1s
🔄 BUILD started (total: 14s)
✅ BUILD completed in 25.4s
🔄 POST_BUILD started (total: 39s)
✅ POST_BUILD completed in 23.3s
🔄 COMPLETED started (total: 63s)
✅ COMPLETED completed in 1.1s
🎉 Cod

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [11]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: langgraph_web_search_agent


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [12]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hi, what can you tell me about EKS"})
invoke_response

{'ResponseMetadata': {'RequestId': '07c7e6fc-2678-4c20-93ab-f571cecfa176',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 25 Sep 2025 22:06:44 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '07c7e6fc-2678-4c20-93ab-f571cecfa176',
   'baggage': 'Self=1-68d5bcda-54a230ac60037d1a1f369c48,session.id=c7a08819-24e0-4ba0-af72-b344c0fcd41e',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'c7a08819-24e0-4ba0-af72-b344c0fcd41e',
   'x-amzn-trace-id': 'Root=1-68d5bcda-5de59ba05d3abe845fb42a67;Self=1-68d5bcda-54a230ac60037d1a1f369c48'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'c7a08819-24e0-4ba0-af72-b344c0fcd41e',
 'traceId': 'Root=1-68d5bcda-5de59ba05d3abe845fb42a67;Self=1-68d5bcda-54a230ac60037d1a1f369c48',
 'baggage': 'Self=1-68d5bcda-54a230ac60037d1a1f369c48,session.id=c7a08819-24e0-4ba0-af72-b344c0fcd41e',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': ['"Based on the

## Congrats!

Congrats, you've deployed and invoked you agent hosted on AgentCore Runtime. As optional step, you can delete the prerequisite infrastructure created